# Домашнее задание «Продвинутый pandas 2»

### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [34]:
stats = pd.read_csv(r'/Users/MacBook/Desktop/курсы/python_netology/python_da/Pandas/lesson 2/ml-latest-small/ratings.csv')
stats.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [33]:
top_users = stats.groupby('userId').count()
top_users = top_users[(top_users.rating > 100)].drop(['movieId', 'timestamp','rating'], axis=1).reset_index()
top_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   userId  258 non-null    int64
dtypes: int64(1)
memory usage: 2.1 KB


In [43]:
stats_top=top_users.merge(stats)
stats.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [38]:
stats_top.userId.nunique()

258

In [39]:
grouped = stats_top.groupby('userId').agg(['min', 'max'])['timestamp'].reset_index()
grouped.head()

,userId,min,max
0,4,949778714,949982274
1,8,1154389340,1154474527
2,15,997937239,1469330735
3,17,1127468587,1127476640
4,19,855190091,855195373


In [40]:
grouped['ttl'] = grouped['max']-grouped['min']
grouped.head()

,userId,min,max,ttl
0,4,949778714,949982274,203560
1,8,1154389340,1154474527,85187
2,15,997937239,1469330735,471393496
3,17,1127468587,1127476640,8053
4,19,855190091,855195373,5282


In [41]:
grouped.ttl.mean()

40080507.4496124

### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [44]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)


auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)


air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)


client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [54]:
full_revenue = rzd.merge(auto, how='outer').merge(air, how='outer').fillna(0)
full_revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [55]:
full_revenue.merge(client_base, how='outer')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

#### Ответ
- Связь можно установить по userID и времени покупки/визита. Далее по этим параметрам подставить координаты пользователя. Возможно нужно будет учитывать, что время получения информации о местоположении не будет точно соответствовать времени покупки/визита, тогда придется задать какой-то допустимый интервал времени, в рамках которого связка будет осуществляться (например 10 мин)

- Необходимо учитывать особенность данных во время join'ов, чтобы не произошло задвоения/потери/искажения информации. Учитывая допустимый инервал времени, возможна ситуация, когда на одну покупку/визит может подойти несколько наборов координат, тогда следует выбирать тот, время получения которого наиболее близко к покупке/визиту. Проверить, что на одну покупку/визит не приходится более 1 набора координат

- Получить информацию о том, откуда пользователи чаще всего совершают покупки/заходят к нам, в каких городах/районах чаще всего пользуются нашим сервисом. Какие разделы/какие товары популярны в каких районах/городах. Если у нас есть физические магазины, можно присылать пользователю рекламные сообщения, когда он находится недалеко от наших точек. 